In [2]:
import numpy as np
### https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
from sklearn.linear_model import Lasso

In [53]:
#https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_dense_vs_sparse_data.html#sphx-glr-auto-examples-linear-model-plot-lasso-dense-vs-sparse-data-py

from time import time
from scipy import sparse
from scipy import linalg

from sklearn.datasets.samples_generator import make_regression
from sklearn.linear_model import Lasso

from math import log


# #############################################################################
# The two Lasso implementations on Dense data
print("--- Dense matrices")

X, y = make_regression(n_samples=200, n_features=5000, random_state=0)
X_sp = sparse.coo_matrix(X)

def alpha_test(X, y, target, effort=5):
    Low = 1
    alpha = 1
    High = 1
    lassy = Lasso(alpha=alpha, fit_intercept=False, max_iter=10**effort)
    coef = np.count_nonzero(lassy.fit(X,y).coef_)
    while coef != target:
        if coef < target:
            # Alpha needs to shrink to allow more coefficients
            High = alpha
            if alpha > Low:
                alpha = (alpha + Low)/2
            else:
                alpha /= 2
                Low = alpha
        else:
            # Alpha needs to grow to restrict the number of coefficients
            Low = alpha
            if alpha < High:
                alpha = (alpha + High)/2
            else:
                alpha *= 3
                High = alpha
            
        if abs(log(abs(Low-High))) > 2*effort:
            print(f"With {coef} nonzero coefficients instead of {target}, we give up!")
            break
        lassy = Lasso(alpha=alpha, fit_intercept=False, max_iter=10**effort)
        coef = np.count_nonzero(lassy.fit(X,y).coef_)
    return alpha

sparse_lasso = Lasso(alpha=alpha, fit_intercept=False, max_iter=1000)
dense_lasso = Lasso(alpha=alpha, fit_intercept=False, max_iter=1000)

t0 = time()
splasso = sparse_lasso.fit(X_sp, y)
print("Sparse Lasso done in %fs" % (time() - t0))

t0 = time()
delasso = dense_lasso.fit(X, y)
print("Dense Lasso done in %fs" % (time() - t0))

print("Distance between coefficients : %s"
      % linalg.norm(sparse_lasso.coef_ - dense_lasso.coef_))


A = [coef for coef in splasso.coef_ if coef]
print(np.count_nonzero(splasso.coef_), np.count_nonzero(delasso.coef_))

--- Dense matrices
Sparse Lasso done in 0.979198s
Dense Lasso done in 0.826327s
Distance between coefficients : 5.126694888601266e-11
1893 1893


In [57]:
for target in range(1,12):
    print(target, alpha_test(X, y, target, effort=6))

1 96.1875
2 91.125
3 81
4 60.64453125
5 57.375
6 54.0
7 47.25
8 46.40625
9 27
10 1
11 0.008880615234375


In [9]:
import numpy as np

matrix = np.array(
    [
        [0, 1 ,0 ,0],
        [0, 0, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 2, 0],
        [1, 0, 0, 1]
    ])

print(np.linalg.matrix_rank(matrix))
matrix.shape

3


(8, 4)

In [22]:
import pandas as pd
df = pd.DataFrame(matrix)
print(df.shape)
cols = [i for i in range(df.shape[1]) if [0,1,0,1][i]]
df[cols]

(8, 4)


,1,3
0,1,0
1,0,0
2,1,0
3,1,0
4,1,0
5,1,0
6,1,0
7,0,1
